In [1]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# https://www.jleague.jp/standings/j1/

teams = [
    'FC東京',
    '鹿島アントラーズ',
    '横浜F・マリノス',
    '川崎フロンターレ',
    'サンフレッチェ広島',
    'セレッソ大阪',
    '北海道コンサドーレ札幌',
    '大分トリニータ',
    '名古屋グランパス',
    '湘南ベルマーレ',
    '浦和レッズ',
    'ヴィッセル神戸',
    '清水エスパルス',
    'ガンバ大阪',
    'ベガルタ仙台',
    'サガン鳥栖',
    '松本山雅FC',
    'ジュビロ磐田'
]

len(teams)

18

In [3]:
import urllib.parse

In [4]:
res = []

# http://footballgeist.com/team/FC%E6%9D%B1%E4%BA%AC&id=player

for team in teams:
    team 
    url = f'http://footballgeist.com/team/{urllib.parse.quote(team)}&id=player'

    df = pd.read_html(url)
    
    data = pd.DataFrame(pd.concat([df[0]['選手一覧'], df[0]['選手一覧.1']], axis=0), columns=['player'])
    data['team'] = team
    
    res.append(data)

In [5]:
result = pd.concat(res)
result.head()

,player,team
0,アウミール モラエス アンドレーデ,FC東京
1,エジミウソン シルバ,FC東京
2,エバウド ドス サントス,FC東京
3,エーベル オリベイラ,FC東京
4,カニーニ,FC東京


In [6]:
len(result)

4098

In [7]:
result.to_csv('result.csv', index=False)

In [8]:
agg = result.groupby('player').count()
agg = agg.query('team > 1').reset_index()

In [9]:
merged = pd.merge(agg, result, on='player', how='left')
merged.head()

,player,team_x,team_y
0,アウグスト ペドロ デ ソウザ,2,鹿島アントラーズ
1,アウグスト ペドロ デ ソウザ,2,川崎フロンターレ
2,アジエル アモリム,2,湘南ベルマーレ
3,アジエル アモリム,2,浦和レッズ
4,アデミウソン,2,横浜F・マリノス


In [25]:
urawa = list(merged[merged['team_y']=='浦和レッズ']['player'].values)

In [10]:
train_data = list(zip(merged['player'], merged['team_y']))

In [11]:
train_data[:10]

[('アウグスト ペドロ デ ソウザ', '鹿島アントラーズ'),
 ('アウグスト ペドロ デ ソウザ', '川崎フロンターレ'),
 ('アジエル アモリム', '湘南ベルマーレ'),
 ('アジエル アモリム', '浦和レッズ'),
 ('アデミウソン', '横浜F・マリノス'),
 ('アデミウソン', 'ガンバ大阪'),
 ('アドリアーノ フェヘイラ', 'セレッソ大阪'),
 ('アドリアーノ フェヘイラ', 'ガンバ大阪'),
 ('アマラオ', 'FC東京'),
 ('アマラオ', '湘南ベルマーレ')]

In [34]:
merged[merged['player']=='田中 マルクス闘莉王']

,player,team_x,team_y
1331,田中 マルクス闘莉王,3,サンフレッチェ広島
1332,田中 マルクス闘莉王,3,名古屋グランパス
1333,田中 マルクス闘莉王,3,浦和レッズ


# https://ai-lab.lapras.com/nlp/poincare-embeddings/

In [12]:
# https://github.com/mukund109/word-mesh/issues/1
# https://qiita.com/wwacky/items/3c33fa0add05c601f555
# !pip install plotly==2.7.0

In [13]:
from gensim.models.poincare import PoincareModel
from gensim.viz.poincare import poincare_2d_visualization
from IPython import display
from plotly.offline import init_notebook_mode, iplot
import pandas as pd
 
init_notebook_mode(connected=True)

/Users/shotaroishihara/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [27]:
model = PoincareModel(train_data, size=2, negative=8)
model.train(epochs=800)

In [28]:
import plotly
plotly.__version__

'2.7.0'

In [29]:
relations_set = set(train_data)
figure_title = ''
iplot(poincare_2d_visualization(
    model,
    relations_set,
    figure_title,
    num_nodes=0,
    show_node_labels=[
    'FC東京',
    '鹿島アントラーズ',
    '横浜F・マリノス',
    '川崎フロンターレ',
    'サンフレッチェ広島',
    'セレッソ大阪',
#     '北海道コンサドーレ札幌',
    '大分トリニータ',
    '名古屋グランパス',
    '湘南ベルマーレ',
    '浦和レッズ',
    'ヴィッセル神戸',
    '清水エスパルス',
    'ガンバ大阪',
    'ベガルタ仙台',
    'サガン鳥栖',
    '松本山雅FC',
    'ジュビロ磐田'
] + urawa
))